In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import cv2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import random
import gc
import matplotlib.pyplot as plt
%matplotlib inline
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_dir = '../input/train'
test_dir = '../input/test'

train_dogs = ['../input/train/{}'.format(i) for i in os.listdir(train_dir) if 'dog' in i] #get dog images
train_cats = ['../input/train/{}'.format(i) for i in os.listdir(train_dir) if 'cat'in i] # get cat images

test_imgs = ['../input/train/{}'.format(i) for i in os.listdir(test_dir)]# get test images

train_imgs = train_dogs[:2000]+ train_cats[:2000]# slice the dataset and use 2000 in each class 
random.shuffle(train_imgs) # shuffle it randomly

# clear list that are useless
del train_dogs 
del train_cats

gc.collect() # collect garbage to save memory 

             

In [ ]:
import matplotlib.image as mpimg
for ima in train_imgs[0:3]:
    img = mpimg.imread(ima)
    imgplot = plt.imshow(img)
    plt.show()

In [ ]:
# lets declare our image dimensions 
#we are using coloured images
nrows = 150
ncolumns = 150
channels = 3

In [ ]:
def read_and_process_image(list_of_images):
    x = [] # images
    y = [] #labels
    
    
    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image, cv2.IMREAD_COLOR), (nrows,ncolumns), interpolation = cv2.INTER_CUBIC)) # Read the image
        # get the labels
        if 'dog' in image:
            y.append(1)
        elif 'cat' in image:
            y.append(0)
            
    return x, y

In [ ]:
x, y = read_and_process_image(train_imgs)
x[0]


In [ ]:
y

In [ ]:
plt.figure(figsize=(20,20))
for i in range(5):
    plt.subplot(15/ 5+1,5, i+1)
    plt.imshow(x[i])


In [ ]:
import seaborn as sns
del train_imgs
gc.collect()

#Convert list to numpy array
x = np.array(x)
y = np.array(y)

#lets plot the label to be sure we just have two classes
sns.countplot(y)
plt.title('Lebles for cat and Dogs')

In [ ]:
print('Shape of train images is', x.shape)
print('Shape of labels is', y.shape)


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_val = train_test_split(x,y,test_size = 0.20, random_state = 2)

print('Shape of train images is :',x_train.shape)
print('Shape of Validation images is :',x_test.shape)
print('Shape of labels  is :',y_train.shape)
print('Shape of labels images is :',y_val.shape)

In [ ]:
# get the  length of the train and vaklidation  data 
ntrain = len(x_train)
nval = len(x_test)

# we will use a batch size of 32; Note that batch sixe should be factor of 2;
batch_size = 32

In [ ]:
from  keras import layers 
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array,load_img

In [ ]:
model = models.Sequential() # Creating sequential model
model.add(layers.Conv2D(32,(3,3), activation = 'relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128,(3,3),activation = 'relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))


In [ ]:
# lets see our model 
model.summary()

In [ ]:
# we'll use the RMSprop Optimixers with a learning rate of 0.0001
#we'll use binary_crossentropy loss because its abinary classification
model.compile(loss = 'binary_crossentropy', optimizer = optimizers.RMSprop(lr=1e-4), metrics=['acc'])

In [ ]:
# lets create the augmentation configuration
#this helps prevent overfitting since we areusing a smaal dataset
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range=0.2,
                                  height_shift_range = 0.2,
                                  width_shift_range=0.2,
                                  shear_range = 0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Create the image generators
train_generator = train_datagen.flow(x_train, y_train, batch_size = batch_size)
val_generator =val_datagen.flow(x_test,y_val,batch_size=batch_size)

In [ ]:
# The training part
# we train for 64 epochs with about 100 steps per seconds
history = model.fit_generator(train_generator,
                             steps_per_epoch=ntrain // batch_size,
                             epochs=69,
                             validation_data=val_generator,
                             validation_steps=nval // batch_size)

In [ ]:
# save the model
model.save_weights('model_weights.h5')
model.save('model_keras.h5')

In [ ]:
# lets plot the train ans val curve
#get the details form the history object
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs =range(1, len(acc)+1)

#train and validation accuracy
plt.plot(epochs, acc ,'b', label='Training accuracy')
plt.plot(epochs,val_acc,'r', label= 'validation_accuracy')
plt.plot('Training and Validation loss')
plt.legend()

plt.show()

In [ ]:
#Now lets predict on the first 10 images of the test set
x_test,y_test = read_and_process_image(test_imgs[0:10])
x= np.array(x_test)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
i=0
text_labels = []
plt.figure(figsize = (30,20))
for batch in text_datagen.flow(x,batch_size = 1):
    pred = model.predict(batch)
    if pred >0.5:
        text_labels.append('dog')
    else:
        text_labels.append('cat')
        plt.subplot(5/columns +1, columns, i+1)
        plt.title('this is a'+text_labels[i])
        imgplot= plt.imshow(batch[0])
        i+=1
        if i%10 ==0:
            break
plt.show()